# Was wordt lijst

Hoofdvraag: welke landelijke typologie (SBB-code, VvN of rVvN) wordt onze standaard die we als input voor Veg2Hab gebruiken en waarom?
- rVvN is het nieuwst, en beschrijft als het goed is de meeste vegetatietypen. gecheckt: als iets een SBB of VvN code heeft, heeft het ook altijd een rVvN code. Andersom niet.
- De opdrachtgever gaf als commentaar terug dat zij vinden dat VvN de standaard moet zijn.

#### Openstaande vragen voor Johannes / begeleidingscommissie:
- Waarom bevat kolom `rVvN` waardes 'Niet'? Moeten we die op NA zetten?
- Waarom staan sommige waardes in kolom `VvN` tussen haakjes? 
- Waarom wil opdrachtgever dat VvN de standaard input voor Veg2Hab is, en niet het nieuwere rVvN?
- Vanaf rij 989 bevat VvN geen waardes meer. rVvN wel, en sommige ook een SBB-code. Waarom geen VvN?
- Wat is de 17e kolom?

#### Beantwoorde vragen:

**Vraag**:

**Antwoord:**

## Data inladen

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel('../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx')

/home/jakko/git/veg2hab/.venv/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
df.columns

Index(['rVvN', 'Wetenschappelijke Naam rVvN', 'Nederlandse Naam rVvN',
       'Veldgids RG', 'VvN', 'SBB-code', 'SBB Wet. Naam',
       'Opmerking vertaling',
       'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)',
       'discussiepunten Wwlijst', 'actie nodig', 'principe', 'impact',
       'discussie/huiswerk', 'wie', 'uitkomst', 'openstaand 27-11-2018',
       'Unnamed: 17'],
      dtype='object')

In [4]:
ww = df[['rVvN','VvN','SBB-code', 'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)']].rename(columns={'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)':'Habtype'})

## Volledigheid typologie

rVvN lijkt de meest specifieke vegetatietypologie. ALles met een waarde voor SBB of VvN heeft altijd (1 uitzondering, zie beneden) ook een rVvN code, andersom niet. 

Velden met een SBB code hebben altijd ook een habitattype ingevuld, velden met VvN of rVvN niet.

In [5]:
ww.isna().sum()

rVvN          5
VvN         257
SBB-code    352
Habtype     211
dtype: int64

In [6]:
# Rijen met een waarde voor rVvN hebben NIET altijd een waarde voor VvN of SBB
ww[~ww['rVvN'].isna()].isna().sum()

rVvN          0
VvN         252
SBB-code    348
Habtype     207
dtype: int64

In [8]:
# Rijen met een waarde voor VvN hebben WEL altijd een waarde voor rVvN, maar niet voor SBB
ww[~ww['VvN'].isna()].isna().sum()

rVvN          0
VvN           0
SBB-code    224
Habtype     183
dtype: int64

In [9]:
# Rijen met een waarde voor SBB-code hebben WEL (bijna) altijd een waarde voor rVvN, maar niet voor VvN
ww[~ww['SBB-code'].isna()].isna().sum()

rVvN          1
VvN         129
SBB-code      0
Habtype       0
dtype: int64

In [10]:
ww[~ww['SBB-code'].isna() & ww['rVvN'].isna()]

,rVvN,VvN,SBB-code,Habtype
1216,NaN,NaN,41A3d,H0000


In [11]:
ww['rVvN'].value_counts().value_counts()

count
1     900
2      88
3      31
5       5
4       5
19      1
6       1
Name: count, dtype: int64

In [45]:
ww['VvN'].value_counts().value_counts()

count
1    709
2     80
3     24
4      5
5      4
6      1
Name: count, dtype: int64

In [46]:
ww['SBB-code'].value_counts().value_counts()

count
1    707
3     25
2     24
4     13
5      2
Name: count, dtype: int64

In [12]:
ww[ww.duplicated(keep=False)]

,rVvN,VvN,SBB-code,Habtype
1240,NaN,NaN,NaN,NaN
1241,NaN,NaN,NaN,NaN
1242,NaN,NaN,NaN,NaN
1243,NaN,NaN,NaN,NaN


## Vinden we alle waarden voor rVvN, VvN en SBB terug in de definitietabel?

Alle vegetatietypen die niet automatisch tot H0000 leiden in de was wordt, zouden we willen terugvinden in de definietabel habitattypen.

Dit is ABSOLUUT niet het geval. Voor VvN vind ik (zonder opschoningsstappen) slechts 9 van de honderden waarden terug. SBB iets meer, enkele tientallen.

In [13]:
dt = pd.read_excel('../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls', sheet_name=1)

In [14]:
dt.head()

,Code vegetatietype,wetenschappelijke naam vegetatietype,Nederlandse naam vegetatietype,Code habitat (sub)type,naam habitat(sub)type,Goed / Matig,beperkende criteria,alleen in mozaïek
0,2Aa1,Ruppietum maritimae,Associatie van Snavelruppia,H1130,Estuaria,G,mits in de FGR Getijdengebied en mits gelegen ...,NaN
1,2Aa1,Ruppietum maritimae,Associatie van Snavelruppia,H1140_A,Slik- en zandplaten (getijdengebied),G,mits in de slikkige en fijnzandige delen van F...,NaN
2,2Aa1,Ruppietum maritimae,Associatie van Snavelruppia,H1160,Grote baaien,G,mits in de FGR Getijdengebied en mits in een i...,NaN
3,2Aa1,Ruppietum maritimae,Associatie van Snavelruppia,H1330_A,Schorren en zilte graslanden (buitendijks),G,NaN,alleen in mozaïek met zelfstandige vegetaties ...
4,2Aa2,Ruppietum cirrhosae,Associatie van Spiraalruppia,H1130,Estuaria,G,mits in de FGR Getijdengebied en mits gelegen ...,NaN


In [15]:
dt.isna().sum()

Code vegetatietype                       35
wetenschappelijke naam vegetatietype     35
Nederlandse naam vegetatietype            0
Code habitat (sub)type                    0
naam habitat(sub)type                     0
Goed / Matig                              0
beperkende criteria                     221
alleen in mozaïek                       453
dtype: int64

In [16]:
dt[dt['Code vegetatietype'].isna()].head()

,Code vegetatietype,wetenschappelijke naam vegetatietype,Nederlandse naam vegetatietype,Code habitat (sub)type,naam habitat(sub)type,Goed / Matig,beperkende criteria,alleen in mozaïek
633,NaN,NaN,"bronvegetatie met beekdikkopmos, geveerd dikne...",H7220,Kalktufbronnen,G,mits tufvorming plaatsvindt,NaN
634,NaN,NaN,overige waterplantengemeenschappen,H3260_A,Beken en rivieren met waterplanten (waterranon...,M,NaN,alleen in mozaïek met zelfstandige vegetaties ...
635,NaN,NaN,overige waterplantengemeenschappen,H3260_B,Beken en rivieren met waterplanten (grote font...,M,NaN,alleen in mozaïek met zelfstandige vegetaties ...
636,NaN,NaN,vegetatieloos,H1110_A,Permanent overstroomde zandbanken (getijdengeb...,G,mits in de slikkige en fijnzandige delen van F...,NaN
637,NaN,NaN,vegetatieloos,H1110_B,Permanent overstroomde zandbanken (Noordzee-ku...,G,mits in de grofzandige delen van FGR Getijdeng...,NaN


In [17]:
dt = dt[~dt['Code vegetatietype'].isna()]

In [18]:
# Verwijderen van alle rijen uit de waswordt lijst die vertalen naar H0000. Deze verwacht je niet terug te vinden in de definitietabel
ww2 = ww.__deepcopy__()
ww2 = ww2[ww2['Habtype']!='H0000']
ww2 = ww2[~ww2['Habtype'].isna()]

In [26]:
def disect_vvn(vvn_code: str):
    # klasse
    klasse = None
    try:
        klasse_span = re.search(r'^\d+', vvn_code).span()
        klasse = vvn_code[klasse_span[0]:klasse_span[1]]
    except:
        print('klasse in VvN code not well defined')
    
    # orde en verbond
    orde = None
    try:
        orde = re.findall('[a-zA-Z]+', vvn_code)[0]
    except:
        print('orde in VvN code not well defined')
    
    # associatie
    associatie = None
    try:
        associatie = re.findall('\d+', vvn_code)[1]
    except:
        print('associatie in VvN code not well defined')
    
    # subassociatie
    subassociatie = None
    try:
        subassociatie = re.findall('[a-zA-Z]+', vvn_code)[1]
    except:
        print('subassociatie in VvN code not well defined')
    
    return klasse, orde, associatie, subassociatie


def clean_vegcode(vegcode: str):
    k, o, a, s = disect_vvn(vegcode)

    # verwijder leading zeros uit klasse 
    k = str(int(k))
    
    # orde en verbond naar lowercase
    o = o.lower()
    
    # verwijder leading zeros uit associatie
    if a is None:
        a = ''
    else:
        a = str(int(a))
    
    # als subassociatie niet aanwezig is, maak er een lege string van.
    if s is None:
        s = ''
    s = s.lower()
        
    return k + o + a + s


def convert_dg_rg_codes(vegcode: str):
    '''
    Alleen gebruikt voor de definitietabel habitattypen. Zet codes voor Rompgemeenschappen en Derivaatgemeenschappen om in normale vegcodes, zodat de waardes matchen met die in de waswordt lijst.
    '''
    return re.sub('-', '', re.sub('\[[\d\w]*\]', '', vegcode))

In [27]:
def transform_vvn(x):
    try:
        return clean_vegcode(x)
    except:
        return None
        
ww2['VvN_transformed'] = ww2['VvN'].apply(lambda x: transform_vvn(x))

klasse in VvN code not well defined
orde in VvN code not well defined
associatie in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
klasse in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well defined
klasse in VvN code not well defined
subassociatie in VvN code not well 

In [28]:
dt['Code vegetatietype'] = dt['Code vegetatietype'].apply(lambda x: convert_dg_rg_codes(x))

In [29]:
def transform_vegcode(x):
    try:
        return clean_vegcode(x)
    except:
        return None
    
dt['vegcode'] = dt['Code vegetatietype'].apply(lambda x: transform_vegcode(x))

subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociatie in VvN code not well defined
subassociat

#### Overlap rVvN en definitietabel

Geen enkele rVvN waarde wordt teruggevonden in de definitietabel. Dat klopt, want die zou alleen VvN en SBB moeten hanteren. 

In [22]:
# Aantal unieke waardes van rVvN.
len(set(ww2['rVvN']))

574

In [23]:
# Aantal hiervan dat hiervan wordt teruggevonden in de definitietabel
len(set(ww2['rVvN']).intersection(set(dt['Code vegetatietype'])))

0

#### Overlap VvN en definitietabel

In [24]:
# Aantal unieke waardes van VvN.
len(set(ww2['VvN']))

456

In [25]:
# Aantal hiervan dat hiervan wordt teruggevonden in de definitietabel
len(set(ww2['VvN']).intersection(set(dt['Code vegetatietype'])))

7

In [30]:
set(ww2['VvN']).intersection(set(dt['Code vegetatietype']))

{'11Ba2', '14RG11', '32Aa', '32Ba', '33Aa', '35Aa3', '43Ab1c', '43Ab1f'}

In [31]:
# Aantal unieke waardes van VvN.
len(set(ww2['VvN_transformed']))

446

In [32]:
len(set(ww2['VvN_transformed']).intersection(set(dt['vegcode'])))

267

#### Overlap SBB en definitietabel

In [33]:
# Ik voeg SBB- toe aan de SBB-code om de waardes te kunnen vergelijken met die uit de definitietabel

def add_SBB_prefix(x):
    if type(x) == str:
        x = 'SBB-' + x
    return x        

ww2['SBB-code'] = ww2['SBB-code'].apply(lambda x: add_SBB_prefix(x))

In [34]:
# Aantal unieke waardes van SBB.
len(set(ww2['SBB-code']))

555

In [35]:
# Aantal hiervan dat hiervan wordt teruggevonden in de definitietabel
len(set(ww2['SBB-code']).intersection(set(dt['Code vegetatietype'])))

0

In [83]:
sbb_herkend = set(ww2['SBB-code']).intersection(set(dt['Code vegetatietype']))

In [84]:
ww2[ww2['SBB-code'].isin(list(sbb_herkend))]

,rVvN,VvN,SBB-code,Habtype
46,r45RG01,(42RG02),SBB-42-d,H2180A mits + H9110m + H9120m + H9190m
47,r45RG06,(42RG02),SBB-42-d,H9110 mits + H9120 mits
253,r09Aa02,09Aa02,SBB-11-h,H7140B
260,r09Aa03b,09Aa03b,SBB-09B2a,H7110A mits + H7140A mits
274,r09RG01,09RG02,SBB-16A-c,H6410m
...,...,...,...,...
1231,r46RG07,NaN,SBB-43-b,H910E0Cm
1232,r46RG08,NaN,SBB-43-d,H2180C mits + H9160Am + H9160Bm + H91F0m
1233,r46RG09,NaN,SBB-42-d,H2180C mits + H9160Am + H9160Bm + H91F0m
1238,Niet overgenomen in Revisie,NaN,SBB-43-e,"toedelen tot revisie-type, anders H0000"


In [85]:
ww2[ww2['SBB-code'] == 'SBB-11-h']

,rVvN,VvN,SBB-code,Habtype
253,r09Aa02,09Aa02,SBB-11-h,H7140B
315,r11Aa02a,11Aa02a,SBB-11-h,H4010A mits + H7110A mits + H7110Bm + H7120 mits
